In [4]:
import argparse
import os
import numpy as np
import math
import sys

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch

os.makedirs("images", exist_ok=True)

In [5]:
epochs = 200
batch_size = 64
lr = 0.0005
n_cpu = 8
latent_dim = 100
img_size = 28 #MNIST的大小为28x28
channels = 1 #输入通道数
n_critic = 5 #训练discriminator次数
clip_value = 0.01 #梯度裁剪
sample_interval = 400 #生成图片保存间隔次数
img_shape = (channels,img_size,img_size) #图片大小 1x28x28
lambda_gp = 10 # Loss weight for gradient penalty
b1=0.5 #Adam的第一个参数
b2=0.999 #Adam的第二个参数


In [6]:
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
            #将数据变形以喂入模型       将数据变为Tensor，并将数据归一化到[0,1]  将数据以0.5均值 0.5方差标准化
        ),
    ),
    batch_size=batch_size,
    shuffle=True,
)

In [9]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            #BatchNorm1d用于对输入参数进行归一化
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            #激活函数使用LeakyReLU
            return layers

        self.model = nn.Sequential(
            #先验z的大小为64x100
            *block(latent_dim, 128, normalize=False),
            #False可能因为输入已经是正态分布了，所以不需要归一化
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            #np.prod计算图片大小 28x28=784
            nn.Tanh()
            #最后使用Tanh激活函数
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.shape[0], *img_shape)
        return img


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
        )

    def forward(self, img):
        img_flat = img.view(img.shape[0], -1)
        validity = self.model(img_flat)
        return validity

In [6]:
generator = Generator()
discriminator = Discriminator()

In [8]:
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))


In [9]:
Tensor = torch.FloatTensor

In [10]:
def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
    alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = D(interpolates)
    fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    # Get gradient w.r.t. interpolates
    gradients = autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

In [11]:
batches_done = 0
for epoch in range(epochs):
    for i, (imgs, _) in enumerate(dataloader):

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))
        #64x100

        # Generate a batch of images
        fake_imgs = generator(z)

        # Real images
        real_validity = discriminator(real_imgs)
        # Fake images
        fake_validity = discriminator(fake_imgs)
        # Gradient penalty
        gradient_penalty = compute_gradient_penalty(discriminator, real_imgs.data, fake_imgs.data)
        # Adversarial loss
        d_loss = -torch.mean(real_validity) + torch.mean(fake_validity) + lambda_gp * gradient_penalty
        #        D(x)表示真实数据分布的均值       D(G(z))表示虚假数据分布的均值

        d_loss.backward()
        optimizer_D.step()

        optimizer_G.zero_grad()

        # Train the generator every n_critic steps
        if i % n_critic == 0:

            # -----------------
            #  Train Generator
            # -----------------

            # Generate a batch of images
            fake_imgs = generator(z)
            # Loss measures generator's ability to fool the discriminator
            # Train on fake images
            fake_validity = discriminator(fake_imgs)
            g_loss = -torch.mean(fake_validity)
            #    D(G(z))表示虚假数据分布的均值

            g_loss.backward()
            optimizer_G.step()

            print(
                "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
                % (epoch, epochs, i, len(dataloader), d_loss.item(), g_loss.item())
            )

            if batches_done % sample_interval == 0:
                save_image(fake_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)

            batches_done += n_critic

[Epoch 0/200] [Batch 0/938] [D loss: 8.153281] [G loss: 0.018273]
[Epoch 0/200] [Batch 5/938] [D loss: -6.761726] [G loss: -0.028216]
[Epoch 0/200] [Batch 10/938] [D loss: -31.071722] [G loss: -0.462318]
[Epoch 0/200] [Batch 15/938] [D loss: -33.958752] [G loss: -0.854936]
[Epoch 0/200] [Batch 20/938] [D loss: -33.830109] [G loss: -1.402647]
[Epoch 0/200] [Batch 25/938] [D loss: -33.386955] [G loss: -2.067870]
[Epoch 0/200] [Batch 30/938] [D loss: -32.149384] [G loss: -2.877134]
[Epoch 0/200] [Batch 35/938] [D loss: -31.321884] [G loss: -3.923843]
[Epoch 0/200] [Batch 40/938] [D loss: -29.754765] [G loss: -5.145681]
[Epoch 0/200] [Batch 45/938] [D loss: -29.214598] [G loss: -6.491386]
[Epoch 0/200] [Batch 50/938] [D loss: -27.333214] [G loss: -7.815718]
[Epoch 0/200] [Batch 55/938] [D loss: -24.942806] [G loss: -9.666124]
[Epoch 0/200] [Batch 60/938] [D loss: -22.549376] [G loss: -11.803108]
[Epoch 0/200] [Batch 65/938] [D loss: -19.860069] [G loss: -13.872845]
[Epoch 0/200] [Batch 70/

[Epoch 0/200] [Batch 590/938] [D loss: -4.110224] [G loss: -0.781092]
[Epoch 0/200] [Batch 595/938] [D loss: -4.470095] [G loss: 0.862268]
[Epoch 0/200] [Batch 600/938] [D loss: -4.816263] [G loss: 0.066480]
[Epoch 0/200] [Batch 605/938] [D loss: -4.284778] [G loss: -0.884019]
[Epoch 0/200] [Batch 610/938] [D loss: -4.365026] [G loss: 1.393659]
[Epoch 0/200] [Batch 615/938] [D loss: -4.350036] [G loss: -0.215776]
[Epoch 0/200] [Batch 620/938] [D loss: -4.623829] [G loss: -0.887355]
[Epoch 0/200] [Batch 625/938] [D loss: -4.380610] [G loss: -0.648452]
[Epoch 0/200] [Batch 630/938] [D loss: -3.953261] [G loss: 0.618700]
[Epoch 0/200] [Batch 635/938] [D loss: -4.141056] [G loss: 1.230774]
[Epoch 0/200] [Batch 640/938] [D loss: -3.794805] [G loss: -1.618883]
[Epoch 0/200] [Batch 645/938] [D loss: -4.621946] [G loss: 1.313064]
[Epoch 0/200] [Batch 650/938] [D loss: -4.115214] [G loss: -1.148575]
[Epoch 0/200] [Batch 655/938] [D loss: -3.668078] [G loss: -1.454270]
[Epoch 0/200] [Batch 660/9

[Epoch 1/200] [Batch 245/938] [D loss: -4.010134] [G loss: 2.561426]
[Epoch 1/200] [Batch 250/938] [D loss: -4.991811] [G loss: 2.318756]
[Epoch 1/200] [Batch 255/938] [D loss: -3.734949] [G loss: 0.291211]
[Epoch 1/200] [Batch 260/938] [D loss: -4.408527] [G loss: -0.343700]
[Epoch 1/200] [Batch 265/938] [D loss: -4.403366] [G loss: -0.874195]
[Epoch 1/200] [Batch 270/938] [D loss: -4.405925] [G loss: -1.975363]
[Epoch 1/200] [Batch 275/938] [D loss: -3.944935] [G loss: -0.527051]
[Epoch 1/200] [Batch 280/938] [D loss: -4.394614] [G loss: 0.533333]
[Epoch 1/200] [Batch 285/938] [D loss: -4.657718] [G loss: 2.766235]
[Epoch 1/200] [Batch 290/938] [D loss: -4.582914] [G loss: 4.416856]
[Epoch 1/200] [Batch 295/938] [D loss: -4.824152] [G loss: 5.138273]
[Epoch 1/200] [Batch 300/938] [D loss: -5.112319] [G loss: 2.842362]
[Epoch 1/200] [Batch 305/938] [D loss: -4.083192] [G loss: 3.167732]
[Epoch 1/200] [Batch 310/938] [D loss: -4.100927] [G loss: 2.541676]
[Epoch 1/200] [Batch 315/938] 

[Epoch 1/200] [Batch 840/938] [D loss: -4.053011] [G loss: -0.155877]
[Epoch 1/200] [Batch 845/938] [D loss: -4.576717] [G loss: -0.019496]
[Epoch 1/200] [Batch 850/938] [D loss: -3.562446] [G loss: -0.739231]
[Epoch 1/200] [Batch 855/938] [D loss: -3.912950] [G loss: -0.618769]
[Epoch 1/200] [Batch 860/938] [D loss: -3.680362] [G loss: -0.235006]
[Epoch 1/200] [Batch 865/938] [D loss: -4.182323] [G loss: -0.469759]
[Epoch 1/200] [Batch 870/938] [D loss: -3.730965] [G loss: -0.580709]
[Epoch 1/200] [Batch 875/938] [D loss: -3.634390] [G loss: -0.713975]
[Epoch 1/200] [Batch 880/938] [D loss: -3.625324] [G loss: -0.295707]
[Epoch 1/200] [Batch 885/938] [D loss: -3.840446] [G loss: -1.412106]
[Epoch 1/200] [Batch 890/938] [D loss: -4.075924] [G loss: -0.295880]
[Epoch 1/200] [Batch 895/938] [D loss: -3.897466] [G loss: 0.281325]
[Epoch 1/200] [Batch 900/938] [D loss: -4.318133] [G loss: 1.509998]
[Epoch 1/200] [Batch 905/938] [D loss: -4.072028] [G loss: 0.897396]
[Epoch 1/200] [Batch 91

[Epoch 2/200] [Batch 495/938] [D loss: -4.001455] [G loss: 0.476787]
[Epoch 2/200] [Batch 500/938] [D loss: -4.180937] [G loss: 1.652123]
[Epoch 2/200] [Batch 505/938] [D loss: -4.420997] [G loss: 0.922086]
[Epoch 2/200] [Batch 510/938] [D loss: -4.166523] [G loss: 0.333695]
[Epoch 2/200] [Batch 515/938] [D loss: -4.441631] [G loss: -0.845048]
[Epoch 2/200] [Batch 520/938] [D loss: -3.687627] [G loss: 0.446068]
[Epoch 2/200] [Batch 525/938] [D loss: -3.708224] [G loss: -0.353469]
[Epoch 2/200] [Batch 530/938] [D loss: -4.207756] [G loss: -0.574100]
[Epoch 2/200] [Batch 535/938] [D loss: -4.167302] [G loss: -0.599594]
[Epoch 2/200] [Batch 540/938] [D loss: -4.277151] [G loss: -0.111993]
[Epoch 2/200] [Batch 545/938] [D loss: -3.883804] [G loss: 0.432956]
[Epoch 2/200] [Batch 550/938] [D loss: -3.339489] [G loss: 0.354562]
[Epoch 2/200] [Batch 555/938] [D loss: -4.220334] [G loss: 0.520969]
[Epoch 2/200] [Batch 560/938] [D loss: -3.526762] [G loss: 0.727983]
[Epoch 2/200] [Batch 565/938]

...

[Epoch 197/200] [Batch 105/938] [D loss: -0.189779] [G loss: -3.022897]
[Epoch 197/200] [Batch 110/938] [D loss: -0.379442] [G loss: -2.991611]
[Epoch 197/200] [Batch 115/938] [D loss: -0.368498] [G loss: -1.464704]
[Epoch 197/200] [Batch 120/938] [D loss: -0.696608] [G loss: -1.383948]
[Epoch 197/200] [Batch 125/938] [D loss: -0.328401] [G loss: -0.678922]
[Epoch 197/200] [Batch 130/938] [D loss: -0.221857] [G loss: -0.377816]
[Epoch 197/200] [Batch 135/938] [D loss: -1.083594] [G loss: -0.388808]
[Epoch 197/200] [Batch 140/938] [D loss: -0.561441] [G loss: 0.280648]
[Epoch 197/200] [Batch 145/938] [D loss: -0.297044] [G loss: 0.228432]
[Epoch 197/200] [Batch 150/938] [D loss: -0.068113] [G loss: 0.074339]
[Epoch 197/200] [Batch 155/938] [D loss: -0.594003] [G loss: -1.601786]
[Epoch 197/200] [Batch 160/938] [D loss: -0.523035] [G loss: -3.297438]
[Epoch 197/200] [Batch 165/938] [D loss: -0.586560] [G loss: -3.220145]
[Epoch 197/200] [Batch 170/938] [D loss: -0.703190] [G loss: -1.574

[Epoch 197/200] [Batch 680/938] [D loss: -0.217821] [G loss: -3.207929]
[Epoch 197/200] [Batch 685/938] [D loss: -0.388241] [G loss: -2.815431]
[Epoch 197/200] [Batch 690/938] [D loss: -0.728741] [G loss: -1.472892]
[Epoch 197/200] [Batch 695/938] [D loss: -0.747281] [G loss: -1.091660]
[Epoch 197/200] [Batch 700/938] [D loss: -0.250048] [G loss: -0.644086]
[Epoch 197/200] [Batch 705/938] [D loss: -0.469584] [G loss: -1.403409]
[Epoch 197/200] [Batch 710/938] [D loss: -0.368379] [G loss: -0.694114]
[Epoch 197/200] [Batch 715/938] [D loss: -0.469377] [G loss: 0.295885]
[Epoch 197/200] [Batch 720/938] [D loss: -0.225155] [G loss: -0.133814]
[Epoch 197/200] [Batch 725/938] [D loss: -0.623312] [G loss: 0.414873]
[Epoch 197/200] [Batch 730/938] [D loss: -0.814473] [G loss: -0.292043]
[Epoch 197/200] [Batch 735/938] [D loss: -0.365958] [G loss: -1.834665]
[Epoch 197/200] [Batch 740/938] [D loss: -0.461266] [G loss: -2.561005]
[Epoch 197/200] [Batch 745/938] [D loss: -0.629123] [G loss: -2.58

[Epoch 198/200] [Batch 315/938] [D loss: -0.376185] [G loss: 0.533429]
[Epoch 198/200] [Batch 320/938] [D loss: -0.102225] [G loss: 0.593107]
[Epoch 198/200] [Batch 325/938] [D loss: -0.647656] [G loss: -0.680864]
[Epoch 198/200] [Batch 330/938] [D loss: -0.184830] [G loss: 0.399082]
[Epoch 198/200] [Batch 335/938] [D loss: -0.190172] [G loss: 0.170184]
[Epoch 198/200] [Batch 340/938] [D loss: -0.218116] [G loss: -1.197909]
[Epoch 198/200] [Batch 345/938] [D loss: -0.535308] [G loss: -2.309495]
[Epoch 198/200] [Batch 350/938] [D loss: -0.234679] [G loss: -4.027565]
[Epoch 198/200] [Batch 355/938] [D loss: -0.457379] [G loss: -3.398327]
[Epoch 198/200] [Batch 360/938] [D loss: -0.724508] [G loss: -3.322015]
[Epoch 198/200] [Batch 365/938] [D loss: -0.155357] [G loss: -2.741916]
[Epoch 198/200] [Batch 370/938] [D loss: -0.911458] [G loss: -3.103806]
[Epoch 198/200] [Batch 375/938] [D loss: -0.812275] [G loss: -2.454181]
[Epoch 198/200] [Batch 380/938] [D loss: -0.775093] [G loss: -3.1536

[Epoch 198/200] [Batch 890/938] [D loss: -0.414550] [G loss: -4.717860]
[Epoch 198/200] [Batch 895/938] [D loss: -0.452861] [G loss: -4.955559]
[Epoch 198/200] [Batch 900/938] [D loss: -0.095905] [G loss: -6.461851]
[Epoch 198/200] [Batch 905/938] [D loss: -0.600161] [G loss: -3.115857]
[Epoch 198/200] [Batch 910/938] [D loss: -0.411259] [G loss: -1.151147]
[Epoch 198/200] [Batch 915/938] [D loss: -0.444058] [G loss: -0.550860]
[Epoch 198/200] [Batch 920/938] [D loss: -0.558702] [G loss: 0.953088]
[Epoch 198/200] [Batch 925/938] [D loss: -0.095420] [G loss: 0.543915]
[Epoch 198/200] [Batch 930/938] [D loss: -0.112238] [G loss: 0.060519]
[Epoch 198/200] [Batch 935/938] [D loss: -0.631445] [G loss: 0.356984]
[Epoch 199/200] [Batch 0/938] [D loss: -0.536383] [G loss: -0.351919]
[Epoch 199/200] [Batch 5/938] [D loss: -0.189060] [G loss: -1.125048]
[Epoch 199/200] [Batch 10/938] [D loss: -0.295576] [G loss: -2.000739]
[Epoch 199/200] [Batch 15/938] [D loss: -0.429023] [G loss: -2.586105]
[E

[Epoch 199/200] [Batch 525/938] [D loss: 0.002489] [G loss: -1.907518]
[Epoch 199/200] [Batch 530/938] [D loss: -0.591113] [G loss: -1.646459]
[Epoch 199/200] [Batch 535/938] [D loss: -0.360495] [G loss: -1.175277]
[Epoch 199/200] [Batch 540/938] [D loss: -0.645867] [G loss: -0.728856]
[Epoch 199/200] [Batch 545/938] [D loss: -0.720192] [G loss: -0.293347]
[Epoch 199/200] [Batch 550/938] [D loss: -0.425595] [G loss: 0.088489]
[Epoch 199/200] [Batch 555/938] [D loss: -0.010622] [G loss: -0.362748]
[Epoch 199/200] [Batch 560/938] [D loss: -0.372069] [G loss: -0.811361]
[Epoch 199/200] [Batch 565/938] [D loss: -0.590317] [G loss: -1.682639]
[Epoch 199/200] [Batch 570/938] [D loss: -0.774903] [G loss: -0.698647]
[Epoch 199/200] [Batch 575/938] [D loss: -0.176493] [G loss: -0.810486]
[Epoch 199/200] [Batch 580/938] [D loss: -0.329453] [G loss: -0.555015]
[Epoch 199/200] [Batch 585/938] [D loss: -0.248119] [G loss: -1.137679]
[Epoch 199/200] [Batch 590/938] [D loss: -0.726835] [G loss: -0.70

In [11]:
generator = Generator()
x = torch.rand(size=(64,100),dtype=torch.float32)
for layer in generator.model:
    x = layer(x)
    print(layer.__class__.__name__,'output shape: \t',x.shape)

Linear output shape: 	 torch.Size([64, 128])
LeakyReLU output shape: 	 torch.Size([64, 128])
Linear output shape: 	 torch.Size([64, 256])
BatchNorm1d output shape: 	 torch.Size([64, 256])
LeakyReLU output shape: 	 torch.Size([64, 256])
Linear output shape: 	 torch.Size([64, 512])
BatchNorm1d output shape: 	 torch.Size([64, 512])
LeakyReLU output shape: 	 torch.Size([64, 512])
Linear output shape: 	 torch.Size([64, 1024])
BatchNorm1d output shape: 	 torch.Size([64, 1024])
LeakyReLU output shape: 	 torch.Size([64, 1024])
Linear output shape: 	 torch.Size([64, 784])
Tanh output shape: 	 torch.Size([64, 784])


In [12]:
np.prod(img_shape)

784